In [1]:
import tetra.utils.data_loader as dl
import tetra.utils.data_processor as dp
from tetra.utils import viewer
import importlib
import matplotlib.pyplot as plt
import os 
from pathlib import Path

In [2]:
dataset_path = "../dataset_sliced/split_1"
class_file = "../classes.txt"

In [3]:
#Load dataframe of all the bounding boxes
df = dl.df_bb(dataset_path,("train", "val"))
df["x_center"] = df["x_center"].astype(float)
df["y_center"] = df["y_center"].astype(float)
df["width"] = df["width"].astype(float)
df["height"] = df["height"].astype(float)
print(df.head())

                                                file class_id  x_center  \
0  05090244_JPG.rf.9ed537c2be76992d772a66a125df8b...        0  0.237500   
1  MFDC7240_JPG.rf.d65b1e0be5f0322f0cd42be767c08f...        0  0.085938   
2  MFDC7660_JPG.rf.7d859cdd5a4f09d02236f1d71aa62e...        0  0.509375   
3  MFDC7660_JPG.rf.7d859cdd5a4f09d02236f1d71aa62e...        0  0.594531   
4  MFDC7660_JPG.rf.7d859cdd5a4f09d02236f1d71aa62e...        0  0.746094   

   y_center     width    height  split  
0  0.836719  0.475000  0.326562  train  
1  0.659375  0.059375  0.096875  train  
2  0.595313  0.065625  0.056250  train  
3  0.597656  0.051562  0.051562  train  
4  0.625781  0.067187  0.035937  train  


In [6]:
df["area"] = df["width"].astype(float) * df["height"].astype(float)
df["aspect_ratioWH"] = df["width"] / df["height"]
df["aspect_ratioHW"] = df["height"] / df["width"]

In [7]:
n = 200
df_sorted = df.sort_values(by="aspect_ratioHW", ascending=False)
criticals = df_sorted.head(n)

In [11]:
#PRINTING
for _, critical in df_invalid.head(10).iterrows():
    image_filename = critical["file"]
    label_filename = image_filename.replace(".png", ".txt")
    critical_img = os.path.join(dataset_path, critical["split"], "images",image_filename)
    critical_label = os.path.join(dataset_path, critical["split"], "labels",label_filename)
    print(critical_img)
    image = viewer.view_image(critical_img, critical_label,class_file)
    plt.imshow(image)
    plt.title(critical["file"])
    plt.axis("off")
    print(f" area : {critical["area"]}")
    print(f"Aspect ratio H / W {critical["aspect_ratioHW"]}")
    print(f"Aspect ratio H / W {critical["aspect_ratioWH"]}")
    plt.show()

In [9]:
#First
invalid = (
    
        ( 
            (df["aspect_ratioHW"] > 2.75) |
            ( df["aspect_ratioWH"] > 3 )
        
        ) &
    
    (df["area"] < 0.005) &
    (
        (df["x_center"] - df["width"] / 2 < 0.0275) |
        (df["x_center"] + df["width"] / 2 > 0.975) |
        (df["y_center"] - df["height"] / 2 < 0.0275) |
        (df["y_center"] + df["height"] / 2 > 0.975)
    )
)

In [10]:
print(f"Invalid bboxes: {invalid.sum()}")
df_invalid = df[invalid]
df_invalid = df_invalid.sort_values(by="area", ascending=False)

Invalid bboxes: 0


In [21]:
splits = ["val", "train"]
for split in splits: 
    path = Path(dataset_path) / split / "labels"
    
    dp.filter_labels(
        label_dir=path,
        condition_fn=lambda cls, x, y, w, h: not (
            (
            (h / w > 2.75) or
            (w / h > 3)
            ) and
            (h * w < 0.005) and
            (
                (x - w / 2 < 0.0275) or
                (x + w / 2 > 0.975) or
                (y - h / 2 < 0.0275) or
                (y + h / 2 > 0.975)
            )
        )
    )

Deleted empty label: ../dataset_sliced/split_1/val/labels/pptly_bonneveau_1-164-_JPG.rf.8c8898088c9c3d07cd2364054fac0687_1693_1536_0_2176_640.txt
Deleted empty label: ../dataset_sliced/split_1/val/labels/05090225_JPG.rf.d4fa194f1b27b302a439e6ecdebc52c3_2387_2048_1536_2688_2176.txt
Deleted empty label: ../dataset_sliced/split_1/val/labels/pptly_bonneveau_1-168-_JPG.rf.7c4c2237eada664a232aba6789a439ed_345_1536_0_2176_640.txt
Deleted empty label: ../dataset_sliced/split_1/val/labels/pptly_bonneveau_1-15-_JPG.rf.6a49d5cce4518e17b6b4f58f4837909f_414_0_0_640_640.txt
Deleted empty label: ../dataset_sliced/split_1/val/labels/MFDC5921_JPG.rf.eeddd8d270680db785feecf11a5e66e1_1123_5120_872_5760_1512.txt
Deleted empty label: ../dataset_sliced/split_1/val/labels/pptly_bonneveau_1-259-_JPG.rf.ef89618b8e86c279ccfda5a72402a848_354_1536_0_2176_640.txt
Total removed boxes: 14
Total deleted label files: 6
Deleted empty label: ../dataset_sliced/split_1/train/labels/pptly_bonneveau_1-170-_JPG.rf.3ed9e362be

In [22]:
for split in splits:
    img_path = Path(dataset_path) / split / "images"
    label_path = Path(dataset_path) / split / "labels"
    dp.remove_imgs_without_annotations(img_path, label_path)

🗑️ Suppression: pptly_bonneveau_1-164-_JPG.rf.8c8898088c9c3d07cd2364054fac0687_1693_1536_0_2176_640.png (No annotations)
🗑️ Suppression: pptly_bonneveau_1-259-_JPG.rf.ef89618b8e86c279ccfda5a72402a848_354_1536_0_2176_640.png (No annotations)
🗑️ Suppression: MFDC5921_JPG.rf.eeddd8d270680db785feecf11a5e66e1_1123_5120_872_5760_1512.png (No annotations)
🗑️ Suppression: 05090225_JPG.rf.d4fa194f1b27b302a439e6ecdebc52c3_2387_2048_1536_2688_2176.png (No annotations)
🗑️ Suppression: pptly_bonneveau_1-15-_JPG.rf.6a49d5cce4518e17b6b4f58f4837909f_414_0_0_640_640.png (No annotations)
🗑️ Suppression: pptly_bonneveau_1-168-_JPG.rf.7c4c2237eada664a232aba6789a439ed_345_1536_0_2176_640.png (No annotations)
🗑️ Suppression: pptly_bonneveau_1-14-_JPG.rf.6b7ea79bccbdc074e04144f57ee8e4e3_985_0_0_640_640.png (No annotations)
🗑️ Suppression: pptly_bonneveau_1-164-_JPG.rf.8c8898088c9c3d07cd2364054fac0687_1693_1952_0_2592_640.png (No annotations)
🗑️ Suppression: pptly_bonneveau_1-258-_JPG.rf.9673d2c447c39385955ee